In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from wikipedia2vec import Wikipedia2Vec
from tqdm import tqdm
import pickle

In [2]:
# load wikipedia2vec model
en_w2v = Wikipedia2Vec.load("../model/enwiki_20180420_300d.pkl")
ja_w2v = Wikipedia2Vec.load("../model/jawiki_20180420_300d.pkl")

In [3]:
# load word pair dict
pair_df = pd.read_csv("../data/title_pair.csv")
print(len(pair_df))
pair_df.head()

244517


,ja,en
0,ベルギー,Belgium
1,幸福,Happiness
2,ジョージ・ワシントン,George Washington
3,ジャック・バウアー,Jack Bauer
4,ダグラス・アダムズ,Douglas Adams


In [5]:
# transfer word to vector

en_emb_list = np.empty((0, 300))
ja_emb_list = np.empty((0, 300))
tmp_en_emb_list = np.empty((0, 300))
tmp_ja_emb_list = np.empty((0, 300))

for i, (ja_word, en_word) in tqdm(pair_df.iterrows()):
    try:
        en_emb = en_w2v.get_entity_vector(en_word)
        ja_emb = ja_w2v.get_entity_vector(ja_word)
        
        tmp_en_emb_list = np.concatenate([tmp_en_emb_list, [en_emb]], axis=0)
        tmp_ja_emb_list = np.concatenate([tmp_ja_emb_list, [ja_emb]], axis=0)
    except KeyError:
        pass
    
    if i % 5000 is 0:
        en_emb_list = np.concatenate([en_emb_list, tmp_en_emb_list], axis=0)
        ja_emb_list = np.concatenate([ja_emb_list, tmp_ja_emb_list], axis=0)
        tmp_en_emb_list = np.empty((0, 300))
        tmp_ja_emb_list = np.empty((0, 300))

244517it [26:30, 153.75it/s]


In [32]:
en_emb_list.shape

(108297, 300)

In [6]:
# fit transfer matrix
model = LinearRegression()
model.fit(X=en_emb_list, y=ja_emb_list)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
# save model
with open("../model/wikipedia2vec_en2ja_mapping.pkl", 'wb') as f:
    pickle.dump(model, f)

In [3]:
# load model
with open("../model/wikipedia2vec_en2ja_mapping.pkl", 'rb') as f:
    model = pickle.load(f)

In [ ]:
ja_w2v.most_similar(ja_w2v.get_entity('食品添加物'))

In [ ]:
en_w2v.most_similar(en_w2v.get_entity('Food additive'))

In [20]:
# Test
# input English word
input_word = "RWBY"
input_vec = en_w2v.get_entity_vector(input_word)
output_vec = model.predict([input_vec])[0]
ja_w2v.most_similar_by_vector(output_vec)

[(<Entity 近田英紀>, 0.7296705955908958),
 (<Entity 深野洋一>, 0.728355075456518),
 (<Entity ミクロマン21>, 0.721611174953668),
 (<Entity :en:Sgt. Frog>, 0.7168662447066008),
 (<Word モニターグラフィックスデザイン>, 0.7167698553191582),
 (<Entity エウレカセブン グラヴィティボーイズ&リフティングガール>, 0.715540253677978),
 (<Entity 平健史>, 0.7145495263896982),
 (<Entity :en:Donatello (Teenage Mutant Ninja Turtles)>, 0.7137585046649126),
 (<Word テマリロボ>, 0.713039766329493),
 (<Entity やまだたかひろ>, 0.7052753182634007),
 (<Entity コピーロボット>, 0.7052650805121463),
 (<Word ナビ・キャップ>, 0.7051567847717995),
 (<Entity 版権#現代における版権の用法>, 0.7047800743857756),
 (<Entity 勇者伝説ブレイブガム>, 0.7044331375474313),
 (<Entity ミルキーカートゥーン>, 0.7041478723179069),
 (<Entity コレクションシリーズ>, 0.7038231148137338),
 (<Entity 超攻戦士 ザクレス>, 0.7028501549967282),
 (<Entity 攻殻機動隊#技術>, 0.7026071114937864),
 (<Entity :en:Michelangelo (Teenage Mutant Ninja Turtles)>,
  0.7023630858995186),
 (<Entity ロボット漫画>, 0.6989795725250362),
 (<Entity ヘキサギア>, 0.6975453510297268),
 (<Entity :pt:Canal Panda>, 0.6